In [ ]:
import random
import heapq
import pygame
import random


GRID_WIDTH = 15
GRID_HEIGHT = 15
CELL_SIZE = 30  # Size of each cell in pixels


if __name__ == "__main__":
    
    # Number of buildings and emergency services
    num_buildings = 7
    num_emergency_services = 4
    # Load images (building and emergency service)
    images = load_images()

    # Generate the grid
    grid = generate_city_grid(GRID_WIDTH, GRID_HEIGHT, num_buildings, num_emergency_services)

    # Assign clusters and calculate the shortest paths
    shortest_paths = find_all_shortest_paths(grid)


    # Visualize the grid and the paths between buildings and emergency services
    visualize_city_grid_with_offset_paths(grid, images, shortest_paths)

In [ ]:
#sample matrix  for troubleshooting
# grid = matrix=[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
# [3, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
# [3, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
# [3, 0, 2, 0, 1, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
# [3, 3, 0, 1, 0, 0, 3, 3, 0, 0, 0, 0, 2, 0, 0, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 3, 2, 0, 0, 0, 3],
# [3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
# [3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3],
# [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
# [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]

In [ ]:
def generate_city_grid(width, height, num_buildings, num_emergency_services):
    """
    Generates the city grid randomly. Places a given number of buildings and emergency services,
    and fills the rest of the grid with roads. Buildings are only placed in alternate rows.
    """
    print(f"Generating grid with {num_buildings} buildings and {num_emergency_services} emergency services...")
    total_cells = width * height
    remaining_cells = total_cells - num_buildings - num_emergency_services

    # Ensure we have enough room for roads and intersections
    if remaining_cells <= 0:
        raise ValueError("Too many buildings and emergency services for the grid size!")

    # Create the initial grid with roads (0) as placeholders
    grid = [[0] * width for _ in range(height)]

    # Determine which rows can have buildings (alternate rows: 0, 2, 4, ...)
    building_rows = [i for i in range(height) if i % 2 != 0]

    # Create a pool of possible positions for buildings and emergency services in the allowed rows
    available_positions = [(row, col) for row in building_rows for col in range(width)]

    # Check if there are enough positions to place all buildings and emergency services
    if len(available_positions) < num_buildings + num_emergency_services:
        raise ValueError("Not enough space to place all buildings and emergency services in alternate rows.")

    # Randomly select positions for buildings and emergency services
    selected_positions = random.sample(available_positions, num_buildings + num_emergency_services)

    # Assign the first 'num_buildings' positions to buildings (1) and the rest to emergency services (2)
    for idx, (row, col) in enumerate(selected_positions):
        if idx < num_buildings:
            grid[row][col] = 1  # Mark this spot with a building
        else:
            grid[row][col] = 2  # Mark this spot with an emergency service

    # Add intersections in columns randomly where possible
    for col in range(width):
        possible_rows = [row for row in range(height) if grid[row][col] == 0]
        if possible_rows:
            selected_row = random.choice(possible_rows)
            grid[selected_row][col] = 3  # Set this cell as an intersection

    # Increase the grid size to add borders of intersections
    width += 2
    height += 2

    # Create the new grid with intersections on the borders
    new_grid = [[3] * width]  # Top row of intersections
    for row in grid:
        new_grid.append([3] + row + [3])  # Add intersection columns to the left and right of each row
    
    new_grid.append([3] * width)  # Bottom row of intersections

    print("Grid generated:")
    for row in new_grid:
        print(row)
    
    return new_grid


In [ ]:
def get_neighbors(grid, node):
    current, prev_direction = node
    x, y = current
    neighbors = []

    # Define possible moves
    directions = {
        "up": (0, -1),
        "down": (0, 1),
        "left": (-1, -0),
        "right": (1, 0)
    }

    # Check the cell value at the current position
    if grid[y][x] == 3:
        # Allowed all directions for grid cell with value 3
        allowed_directions = ["up", "down", "left", "right"]
    elif prev_direction is None:
        # Starting position, can move only up or down
        allowed_directions = ["up", "down"]
    elif prev_direction in ["up", "right"]:
        # Previous direction was up or right, can only move right
        allowed_directions = ["right"]
    elif prev_direction in ["down", "left"]:
        # Previous direction was down or left, can only move left
        allowed_directions = ["left"]
    else:
        allowed_directions = []  # default empty list if no directions are allowed

    # Calculate neighbors based on allowed directions
    for direction in allowed_directions:
        dx, dy = directions[direction]
        new_x, new_y = x + dx, y + dy

        # Ensure new coordinates are within grid bounds
        if 0 <= new_x < len(grid[0]) and 0 <= new_y < len(grid):
            neighbors.append(((new_x, new_y), direction))
    return neighbors

def manhattan_distance(node1, node2):
    # Extract coordinates from each node
    (x1, y1), _ = node1  # node1 has (coordinates, direction)
    (x2, y2), _ = node2  # node2 has (coordinates, direction)
    return abs(x1 - x2) + abs(y1 - y2)


def a_star(grid, start, goal):
    # Initialize open and closed lists
    open_list = []
    closed_list = set()

    # Initialize start node correctly
    start_node = start  # (coordinates, direction)
    g_costs = {start_node: 0}
    f_costs = {start_node: manhattan_distance(start_node, goal)}

    # Add start node to open list with initial f-cost
    heapq.heappush(open_list, (f_costs[start_node], start_node))

    # Main loop
    while open_list:
        # Get node with lowest f-cost
        _, current_node = heapq.heappop(open_list)

        # Check if goal is reached
        if current_node[0] == goal[0]:  # Compare only coordinates, ignoring direction
            print('done')
            return 0  # Implement reconstruct_path to trace back the path

        closed_list.add(current_node)

        # Get neighbors
        for neighbor in get_neighbors(grid, current_node):
            if neighbor in closed_list:
                continue
            print("Current node:", current_node)
            print("Neighbors:", get_neighbors(grid, current_node))

            tentative_g_cost = g_costs[current_node] + 1

            if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = tentative_g_cost
                h_cost = manhattan_distance(neighbor, goal)
                f_cost = tentative_g_cost + h_cost
                f_costs[neighbor] = f_cost
                heapq.heappush(open_list, (f_cost, neighbor))

    # If no path found
    print('no path found')
    return None



In [ ]:

def is_intersection_and_above_below(current,grid,goal):
   return (grid[current[1]][current[0]]==3  and 
           ((current[1] == goal[1] - 1) or (current[1] == goal[1] + 1)))


def a_star(grid, start, goal):
    # Initialize open and closed lists

    open_list = []
    closed_list = set()
    came_from = {}  # Dictionary to store the parent of each node for path reconstruction

    # Initialize start node
    start_node = start  # (coordinates, direction)
    g_costs = {start_node: 0}
    f_costs = {start_node: manhattan_distance(start_node, goal)}

    # Add start node to open list with initial f-cost
    heapq.heappush(open_list, (f_costs[start_node], start_node))

    # Main loop
    while open_list:
        # Get node with lowest f-cost
        _, current_node = heapq.heappop(open_list)

        current, prev_direction = current_node
        goal_position, _ = goal  # Unpack goal coordinates and ignore the direction
        # print('goal is ',prev_direction)
        if (current[0] == goal_position[0]  and 
            ((current[1] + 1 == goal_position[1] and prev_direction == 'right') or 
            (current[1] - 1 == goal_position[1] and prev_direction == 'left') or 
            is_intersection_and_above_below(current, grid, goal_position))):
            # print(current_node)  # Compare only coordinates, ignoring direction
            return reconstruct_path(came_from, current_node)  # Reconstruct the path from start to goal

        # Add current node to closed list
        closed_list.add(current_node)

        # Get neighbors
        neighbors = get_neighbors(grid, current_node)

        for neighbor in neighbors:
            if neighbor in closed_list:
                continue

            tentative_g_cost = g_costs[current_node] + 1

            if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                # Update costs and add to open list
                came_from[neighbor] = current_node  # Track the parent of the neighbor for path reconstruction
                g_costs[neighbor] = tentative_g_cost
                h_cost = manhattan_distance(neighbor, goal)
                f_cost = tentative_g_cost + h_cost
                f_costs[neighbor] = f_cost
                heapq.heappush(open_list, (f_cost, neighbor))

    # If no path found
    print(reconstruct_path(came_from, current_node))
    return None

def reconstruct_path(came_from, current_node):
    path = [current_node]
    while current_node in came_from:
        current_node = came_from[current_node]
        path.append(current_node)
    path.reverse()  # Reverse the path to get it from start to goal
    return path


def a_star_multiple_goals(grid, start):
    # Identify all emergency services (goals)
    emergencies = []
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == 2:  # Emergency service marked as 2
                emergencies.append(((x, y), None))  # Add as (coordinates, direction)

    print(f"Found {len(emergencies)} emergency services.")

    shortest_path = None
    shortest_length = float('inf')

    # Run A* for each emergency service location
    for goal in emergencies:
        path = a_star(grid, start, goal)
        if path and len(path) < shortest_length:
            shortest_path = path
            shortest_length = len(path)

    # Return the shortest path found, if any
    if shortest_path:
        print(f"Shortest path to an emergency service: {shortest_path} with length {shortest_length}")
    else:
        print("No path found to any emergency service.")
    
    return shortest_path

In [ ]:
def manhattan_distance(node1, node2):
    # print(node1,node2)
    # Extract coordinates from each node
    (x1, y1), _ = node1  # node1 has (coordinates, direction)
    (x2, y2), _ = node2  # node2 has (coordinates, direction)
    return abs(x1 - x2) + abs(y1 - y2)

def find_all_shortest_paths(grid):
    """
    Finds the shortest path from each building to the nearest emergency service.
    Returns an array of shortest paths for each building.
    """
    buildings = []  # Array to store building positions
    shortest_paths = []  # Array to store the shortest paths for each building

    # Step 1: Locate all buildings
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == 1:  # Building marked as 1
                buildings.append((x, y))

    print(f"Found {len(buildings)} buildings.")

    # Step 2: For each building, use BFS to find the shortest path to the nearest emergency service
    for building in buildings:
        print(f"Calculating shortest path for building at {building}...")
        shortest_path = a_star_multiple_goals(grid, (building,None))  # Use BFS to get shortest path for the building
        if shortest_path:
            shortest_paths.append(shortest_path)  # Add the path to the list if found
        else:
            print(f"No path found to any emergency service for building at {building}.")

    return shortest_paths


In [ ]:

def visualize_city_grid_with_offset_paths(grid, images, paths):
    """ 
    Visualizes the grid with buildings, emergency services, and roads using PyGame.
    Also visualizes the shortest paths between buildings and emergency services, each in a unique color with slight offsets for overlapping paths.
    """
    pygame.init()  # Initialize PyGame
    screen = pygame.display.set_mode((len(grid) * CELL_SIZE, len(grid) * CELL_SIZE))  # Set screen size
    pygame.display.set_caption("City Layout with Offset Paths")  # Window title

    clock = pygame.time.Clock()  # Set up the clock for controlling frame rate
    running = True  # Game loop control variable

    # Define a list of colors for the paths, avoiding green (the intersection color)
    path_colors = [(255, 0, 0), (0, 0, 255), (255, 165, 0), (255, 255, 0), 
                   (0, 255, 255), (255, 0, 255), (128, 0, 128), (0, 128, 128), 
                   (255, 192, 203), (0, 100, 0), (139, 69, 19), (75, 0, 130), 
                   (128, 128, 128), (100, 149, 237), (199, 21, 133)]

    path_thickness = 6  # Set the thickness for paths
    max_offset = 3  # Set maximum offset for paths to avoid overflowing outside the cell
    road_color = (169, 169, 169)  # Gray for roads
    road_border_color = (255, 255, 255)  # White for road borders
    border_thickness = 3  # Thickness of the white border

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Quit the game if the close button is pressed
                running = False

        screen.fill((128, 128, 128))  # Gray background

        # Draw the grid using images (building, emergency services, etc.)
        for y in range(len(grid)):
            for x in range(len(grid)):
                cell_value = grid[y][x]
                if cell_value in images:  # If the cell has a corresponding image (building or emergency)
                    screen.blit(images[cell_value], (x * CELL_SIZE, y * CELL_SIZE))  # Draw the image
                elif cell_value == 3:  # If the cell is an intersection
                    pygame.draw.rect(screen, (52, 251, 152), (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))  # Draw intersections in green
                elif cell_value == 0:  # Draw roads with a border
                    # Draw the white border for the road
                    pygame.draw.rect(screen, road_border_color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                    # Draw the inner part of the road (smaller than the cell to show the border)
                    pygame.draw.rect(screen, road_color, 
                                     (x * CELL_SIZE + border_thickness, 
                                      y * CELL_SIZE + border_thickness, 
                                      CELL_SIZE - 2 * border_thickness, 
                                      CELL_SIZE - 2 * border_thickness))

        # Draw the paths in different colors with slight offsets for overlapping paths
        for i, path in enumerate(paths):
            path_color = path_colors[i % len(path_colors)]  # Cycle through colors if there are more paths than colors
            for (x, y) in path:
                if grid[y][x] != 1 and grid[y][x] != 2:  # Only color if it's a road (0), to avoid coloring buildings, emergency services, or intersections
                    # Calculate the offset based on the index of the path to avoid overlap
                    offset_x = (i % max_offset) * 2  # Offset in x-direction
                    offset_y = (i % max_offset) * 2  # Offset in y-direction
                    
                    # Ensure the rectangle fits within the cell and apply a small offset
                    pygame.draw.rect(
                        screen, 
                        path_color, 
                        (x * CELL_SIZE + offset_x, y * CELL_SIZE + offset_y, CELL_SIZE - max_offset, CELL_SIZE - max_offset),
                        path_thickness
                    )

        pygame.display.flip()  # Update the display
        clock.tick(30)  # Control the frame rate (30 FPS)
    pygame.quit()  # Properly shut down PyGame



pygame 2.1.0 (SDL 2.0.16, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:

def visualize_city_grid_with_offset_paths(grid, images, paths):
    """ 
    Visualizes the grid with buildings, emergency services, and roads using PyGame.
    Also visualizes the shortest paths between buildings and emergency services, each in a unique color with slight offsets for overlapping paths.
    """
    pygame.init()  # Initialize PyGame
    screen = pygame.display.set_mode((len(grid) * CELL_SIZE, len(grid) * CELL_SIZE))  # Set screen size
    pygame.display.set_caption("City Layout with Offset Paths")  # Window title

    clock = pygame.time.Clock()  # Set up the clock for controlling frame rate
    running = True  # Game loop control variable

    # Define a list of colors for the paths, avoiding green (the intersection color)
    path_colors = [(255, 0, 0), (0, 0, 255), (255, 165, 0), (255, 255, 0), 
                   (0, 255, 255), (255, 0, 255), (128, 0, 128), (0, 128, 128), 
                   (255, 192, 203), (0, 100, 0), (139, 69, 19), (75, 0, 130), 
                   (128, 128, 128), (100, 149, 237), (199, 21, 133)]

    path_thickness = 6  # Set the thickness for paths
    max_offset = 3  # Set maximum offset for paths to avoid overflowing outside the cell
    road_color = (169, 169, 169)  # Gray for roads
    road_border_color = (255, 255, 255)  # White for road borders
    border_thickness = 3  # Thickness of the white border

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Quit the game if the close button is pressed
                running = False

        screen.fill((128, 128, 128))  # Gray background

        # Draw the grid using images (building, emergency services, etc.)
        for y in range(len(grid)):
            for x in range(len(grid)):
                cell_value = grid[y][x]
                if cell_value in images:  # If the cell has a corresponding image (building or emergency)
                    screen.blit(images[cell_value], (x * CELL_SIZE, y * CELL_SIZE))  # Draw the image
                elif cell_value == 3:  # If the cell is an intersection
                    pygame.draw.rect(screen, (52, 251, 152), (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))  # Draw intersections in green
                elif cell_value == 0:  # Draw roads with a border
                    # Draw the white border for the road
                    pygame.draw.rect(screen, road_border_color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                    # Draw the inner part of the road (smaller than the cell to show the border)
                    pygame.draw.rect(screen, road_color, 
                                     (x * CELL_SIZE + border_thickness, 
                                      y * CELL_SIZE + border_thickness, 
                                      CELL_SIZE - 2 * border_thickness, 
                                      CELL_SIZE - 2 * border_thickness))

        # Draw the paths in different colors with slight offsets for overlapping paths
        for i, path in enumerate(paths):
            path_color = path_colors[i % len(path_colors)]  # Cycle through colors if there are more paths than colors
            for (coord, direction) in path:
                x,y=coord
                if grid[y][x] != 1 and grid[y][x] != 2:  # Only color if it's a road (0), to avoid coloring buildings, emergency services, or intersections
                    # Calculate the offset based on the index of the path to avoid overlap
                    offset_x = (i % max_offset) * 2  # Offset in x-direction
                    offset_y = (i % max_offset) * 2  # Offset in y-direction
                    
                    # Ensure the rectangle fits within the cell and apply a small offset
                    pygame.draw.rect(
                        screen, 
                        path_color, 
                        (x * CELL_SIZE + offset_x, y * CELL_SIZE + offset_y, CELL_SIZE - max_offset, CELL_SIZE - max_offset),
                        path_thickness
                    )

        pygame.display.flip()  # Update the display
        clock.tick(30)  # Control the frame rate (30 FPS)
    pygame.quit()  # Properly shut down PyGame



In [ ]:
def load_images():
    """
    Loads images for each type of grid element (building, emergency service, etc.).
    Scales the images to fit within the grid cells.
    """
    print("Loading images...")
    images = {
        1: pygame.image.load('building.jpg'),           # Building image (jpeg format)
        2: pygame.image.load('emergency.png')           # Emergency service image (png format)
        # You can add more images for roads and intersections here
    }
    
    # Scale all images to fit into the grid cells
    for key in images:
        images[key] = pygame.transform.scale(images[key], (CELL_SIZE, CELL_SIZE))
    
    print("Images loaded and scaled.")
    return images

images=load_images()



Loading images...
Images loaded and scaled.


In [55]:
def visualize_city_grid_with_offset_paths(grid, images, paths):
    """ 
    Visualizes the grid with buildings, emergency services, and roads using PyGame.
    Also visualizes the shortest paths between buildings and emergency services, each in a unique color with slight offsets for overlapping paths.
    """
    pygame.init()  # Initialize PyGame
    screen = pygame.display.set_mode((len(grid) * CELL_SIZE, len(grid) * CELL_SIZE))  # Set screen size
    pygame.display.set_caption("City Layout with Offset Paths")  # Window title

    clock = pygame.time.Clock()  # Set up the clock for controlling frame rate
    running = True  # Game loop control variable

    # Define a list of colors for the paths, avoiding green (the intersection color)
    path_colors = [(255, 0, 0), (0, 0, 255), (255, 165, 0), (255, 255, 0), 
                   (0, 255, 255), (255, 0, 255), (128, 0, 128), (0, 128, 128), 
                   (255, 192, 203), (0, 100, 0), (139, 69, 19), (75, 0, 130), 
                   (128, 128, 128), (100, 149, 237), (199, 21, 133)]

    path_thickness = 6  # Set the thickness for paths
    max_offset = 3  # Set maximum offset for paths to avoid overflowing outside the cell
    road_color = (169, 169, 169)  # Gray for roads
    road_border_color = (255, 255, 255)  # White for road borders
    border_thickness = 3  # Thickness of the white border

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Quit the game if the close button is pressed
                running = False

        screen.fill((128, 128, 128))  # Gray background

        # Draw the grid using images (building, emergency services, etc.)
        for y in range(len(grid)):
            for x in range(len(grid)):
                cell_value = grid[y][x]
                if cell_value in images:  # If the cell has a corresponding image (building or emergency)
                    screen.blit(images[cell_value], (x * CELL_SIZE, y * CELL_SIZE))  # Draw the image
                elif cell_value == 3:  # If the cell is an intersection
                    pygame.draw.rect(screen, (52, 251, 152), (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))  # Draw intersections in green
                elif cell_value == 0:  # Draw roads with a border
                    # Draw the white border for the road
                    pygame.draw.rect(screen, road_border_color, (x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE))
                    # Draw the inner part of the road (smaller than the cell to show the border)
                    pygame.draw.rect(screen, road_color, 
                                     (x * CELL_SIZE + border_thickness, 
                                      y * CELL_SIZE + border_thickness, 
                                      CELL_SIZE - 2 * border_thickness, 
                                      CELL_SIZE - 2 * border_thickness))

        # Draw the paths in different colors with slight offsets for overlapping paths
        for i, path in enumerate(paths):
            path_color = path_colors[i % len(path_colors)]  # Cycle through colors if there are more paths than colors
            for (coord, direction) in path:
                x,y=coord
                if grid[y][x] != 1 and grid[y][x] != 2:  # Only color if it's a road (0), to avoid coloring buildings, emergency services, or intersections
                    # Calculate the offset based on the index of the path to avoid overlap
                    offset_x = (i % max_offset) * 2  # Offset in x-direction
                    offset_y = (i % max_offset) * 2  # Offset in y-direction
                    
                    # Ensure the rectangle fits within the cell and apply a small offset
                    pygame.draw.rect(
                        screen, 
                        path_color, 
                        (x * CELL_SIZE + offset_x, y * CELL_SIZE + offset_y, CELL_SIZE - max_offset, CELL_SIZE - max_offset),
                        path_thickness
                    )

        pygame.display.flip()  # Update the display
        clock.tick(30)  # Control the frame rate (30 FPS)
    pygame.quit()  # Properly shut down PyGame
